<a href="https://colab.research.google.com/github/nic-fp/Learning-Machine-Learning/blob/master/Chapter10.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
import tensorflow as tf
import numpy as np

(X_train, y_train), (X_test, y_test) = tf.keras.datasets.mnist.load_data()
X_train = X_train.astype(np.float32).reshape(-1, 28*28) / 255.0
X_test = X_test.astype(np.float32).reshape(-1, 28*28) / 255.0
y_train = y_train.astype(np.int32)
y_test = y_test.astype(np.int32)
X_valid, X_train = X_train[:5000], X_train[5000:]
y_valid, y_train = y_train[:5000], y_train[5000:]
feature_cols = [tf.feature_column.numeric_column("X", shape=[28 * 28])]
dnn_clf = tf.estimator.DNNClassifier(hidden_units=[300,100], n_classes=10,feature_columns=feature_cols)

input_fn = tf.estimator.inputs.numpy_input_fn(
    x={"X": X_train}, y=y_train, num_epochs=40, batch_size=50, shuffle=True)
dnn_clf.train(input_fn=input_fn)


11493376/11490434 [==============================] - 0s 0us/step
INFO:tensorflow:Using default config.
INFO:tensorflow:Using config: {'_model_dir': '/tmp/tmpe6dfs2qh', '_tf_random_seed': None, '_save_summary_steps': 100, '_save_checkpoints_steps': None, '_save_checkpoints_secs': 600, '_session_config': allow_soft_placement: true
graph_options {
  rewrite_options {
    meta_optimizer_iterations: ONE
  }
}
, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_log_step_count_steps': 100, '_train_distribute': None, '_device_fn': None, '_protocol': None, '_eval_distribute': None, '_experimental_distribute': None, '_experimental_max_worker_delay_secs': None, '_service': None, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x7f4bba689080>, '_task_type': 'worker', '_task_id': 0, '_global_id_in_cluster': 0, '_master': '', '_evaluation_master': '', '_is_chief': True, '_num_ps_replicas': 0, '_num_worker_replicas': 1}
Instructions for updating:
Use 

In [0]:
test_input_fn = tf.estimator.inputs.numpy_input_fn(
    x={"X": X_test}, y=y_test, shuffle=False)
eval_results = dnn_clf.evaluate(input_fn=test_input_fn)

INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Starting evaluation at 2019-09-24T21:58:38Z
INFO:tensorflow:Graph was finalized.
Instructions for updating:
Use standard file APIs to check for files with this prefix.
INFO:tensorflow:Restoring parameters from /tmp/tmpx6m7bcv0/model.ckpt-44000
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
INFO:tensorflow:Finished evaluation at 2019-09-24-21:58:38
INFO:tensorflow:Saving dict for global step 44000: accuracy = 0.9811, average_loss = 0.0959083, global_step = 44000, loss = 12.140291
INFO:tensorflow:Saving 'checkpoint_path' summary for global step 44000: /tmp/tmpx6m7bcv0/model.ckpt-44000


In [0]:
import tensorflow as tf
import numpy as np
def reset_graph(seed=42):
    tf.reset_default_graph()
    tf.set_random_seed(seed)
    np.random.seed(seed)

n_inputs= 28*28
n_hidden1=300 
n_hidden2=100
n_outputs= 10
reset_graph()
X= tf.placeholder(tf.float32,shape=(None,n_inputs),name="X")
Y= tf.placeholder(tf.int32, shape=(None), name= "Y")

def neuron_layer(X,n_neurons,name,activation=None):
  with tf.name_scope(name):
    n_inputs = int(X.get_shape()[1])
    stddev= 2/np.sqrt(n_inputs+n_neurons)
    init= tf.truncated_normal((n_inputs,n_neurons), stddev= stddev)
    W= tf.Variable(init, name="kernel")
    b= tf.Variable(tf.zeros([n_neurons]), name= "bias")
    Z = tf.matmul(X,W) + b
    if activation is not None:
      return activation(Z)

    else:
      return(Z)

"""with tf.name_scope("dnn"): #manual version of layerdense
  hidden1 = neuron_layer(X,n_hidden1, name= "hidden1", activation= tf.nn.relu)
  hidden2 = neuron_layer(hidden1,n_hidden2, name= "hidden2", activation= tf.nn.relu)
  logits = neuron_layer(hidden2,n_outputs, name= "outputs")"""
with tf.name_scope("dnn"):
  hidden1 = tf.keras.layers.Dense(X,n_hidden1, name= "hidden1", activation= tf.nn.relu)
  hidden2 = tf.keras.layers.Dense(hidden1,n_hidden2, name= "hidden2", activation= tf.nn.relu)
  logits = tf.keras.layers.Dense(hidden2,n_outputs, name= "outputs")

with tf.name_scope("loss"):
  xentrophy= tf.nn.sparse_softmax_cross_entropy_with_logits(labels=Y, logits=logits)
  loss = tf.reduce_mean(xentrophy,name="loss")

learning_rate= .1
with tf.name_scope("train"):
  optimizer = tf.train.GradientDescentOptimizer(learning_rate)
  training_op = optimizer.minimize(loss)

with tf.name_scope("eval"):
  correct = tf.nn.in_top_k(logits,Y,1)
  accuracy = tf.reduce_mean(tf.cast(correct,tf.float32))

init = tf.global_variables_initializer()
saver= tf.train.Saver()

from tensorflow.examples.tutorials.mnist import input_data
mnist = input_data.read_data_sets("/tmp/data/")

n_epochs = 40
batch_size = 50

with tf.Session() as sess:
  init.run()
  for epoch in range(n_epochs):
    for iteration in range(mnist.train.num_examples // batch_size):
      X_batch, y_batch = mnist.train.next_batch(batch_size)
      sess.run(training_op, feed_dict = {X:X_batch, Y : y_batch})
    acc_train = accuracy.eval(feed_dict={X:X_batch,Y: y_batch})
    acc_val = accuracy.eval(feed_dict={X:mnist.validation.images,Y: mnist.validation.labels})
    print(epoch,"Train accuracy", acc_train, "Val accuracy", acc_val)
  save_path = saver.save(sess, "./my_model_final.ckpt")

TypeError: ignored

In [0]:
with tf.Session() as sess:
    saver.restore(sess, "./my_model_final.ckpt") # or better, use save_path
    X_new_scaled = X_test[200:212]
    Z = logits.eval(feed_dict={X: X_new_scaled})
    y_pred = np.argmax(Z, axis=1)


print("Predicted classes:", y_pred)
print("Actual classes:   ", y_test[200:212])


INFO:tensorflow:Restoring parameters from ./my_model_final.ckpt
Predicted classes: [3 6 1 1 1 3 9 5 2 9 4 5]
Actual classes:    [3 6 1 1 1 3 9 5 2 9 4 5]
